In [ ]:

# Save figures in specific place
knitr::opts_chunk$set(autodep        = TRUE,
                      cache          = FALSE,
                      cache.comments = TRUE,
                      comment = "",

                      # Save figures as pdf ?
                      dev = c("png", "pdf"),

                      # Include code?
                      echo           = TRUE,
                      warning = FALSE, 
                      error          = FALSE,
                      fig.align      = "center",

                      # Path where figures are going to be store pdf single
                      # figures
                      fig.path       = paste0("./figures", "/"),
                      fig.width      = 11,
                      fig.height     = 7,
                      message        = FALSE,
                      warning        = FALSE)

# set to two decimal
options(scipen = 1, digits = 8)

In [ ]:
library(dplyr)
library(nlme)
library(purrr)
library(performance)
library(tibble)
library(here)
library(reactablefmtr)
library(emmeans)
library(car)
library(ggplot2)
library(modelr)
library(knitr)

In [ ]:
# Set knit directory
setwd(here::here())
knitr::opts_knit$set(root.dir = setwd(here::here()))
getwd()

# Load functions and data

In [ ]:
# Load all joined dataset
source("./scripts/code_join_data_full_dataset.R")

In [ ]:
# Load custom made functions
source("./R/functions_models.R")
source("./R/function_plots.R")
source("./R/function_validation_plots.R")
source("./R/function_for_inference_emmeans_and_percentage_diff.R")

# Research question

__Q1: How do plant responses to water and nutrients vary between N-Fixers and non-fixers?__

# Model fitting

In [ ]:
# Create empty list
models_q1 <- list()

In [ ]:
model_q1_total_biomass <- lme(total_biomass ~ nfixer*treatment + init_height,
                                                random = ~1|spcode,             
                                                data = data_for_models)

model_q1_total_biomass <- list(model_q1_total_biomass)
names(model_q1_total_biomass) <- "total_biomass"

In [ ]:
model_q1_rgr <- lme(rgr ~ nfixer*treatment + init_height,
                                                random = ~1|spcode,             
                                                data = data_for_models)

model_q1_rgr <- list(model_q1_rgr)
names(model_q1_rgr) <- "rgr"

In [ ]:

# Log model

## Root-Shoot ratio
model_q1_ratio_log <- lme(log(root_shoot_ratio) ~ nfixer*treatment + init_height,
                                                random = ~1|spcode,             
                                                data = data_for_models)

log_models_q1 <- list(model_q1_ratio_log)

names(log_models_q1) <- "root_shoot_ratio_log"

In [ ]:
# Append log models to model list
models_q1 <- append(model_q1_total_biomass, models_q1)
models_q1 <- append(model_q1_rgr, models_q1)
models_q1 <- append(log_models_q1, models_q1)

In [ ]:
names(models_q1)

# Model Assumptions

## Total Biomass

In [ ]:
validation_plots(models_q1$total_biomass, data = data_for_models,group = "spcode")

## RGR

In [ ]:
validation_plots(models_q1$rgr, data = data_for_models, group = "spcode")

## Root-Shoot ratio

In [ ]:
validation_plots(models_q1$root_shoot_ratio_log, data = data_for_models, 
                group = "spcode")

# Model inference

In [ ]:
## r2 models
models_q1 %>%
    map(., r2_nakagawa) %>%
    unlist() %>%
    kable()

In [ ]:
models_q1 %>%
    map(., AIC) %>%
    unlist()

## Anova tables

All models show a significant two-way interaction between Nfixer and Treatment

In [ ]:
#map(models_q1, ~Anova(.x, type = "III", test.statistic = c("F")))
map(models_q1, ~anova.lme(.x, type = "marginal"))

### Post-Hoc: Tukey's test

#### Total Biomass

In [ ]:
as_tibble(emmeans(models_q1$total_biomass,
        pairwise ~ treatment*nfixer,
        adjust = "tukey"
        )$contrast) %>%
        mutate(across(2:6, round, 6)) %>%
        kable()

In [ ]:
## Treatment effects
emmeans_table_tidy(models_q1$total_biomass,
                        formula = "treatment|nfixer",
                        grouping_var = "nfixer")

#### RGR

In [ ]:
as_tibble(emmeans(models_q1$rgr,
        pairwise ~ treatment*nfixer,
        adjust = "tukey"
        )$contrast) %>%
        mutate(across(2:6, round, 6)) %>%
        kable()

In [ ]:
# Treatment effects
emmeans_table_tidy(models_q1$root_shoot_ratio_log,
                        formula = "treatment|nfixer",
                        grouping_var = "nfixer")

## Boxplots Biomass and growth

In [ ]:
# Step done for getting predictions from models for Q1
data_for_predictions <-
    data_for_models %>%

        rownames_to_column("id") %>%

        # Remove unused variables
        dplyr::select(id, spcode, treatment, nfixer, init_height)

In [ ]:
# Get predictions
string <- c("models_q1")

data_pred_biomass_growth <-

        # Get models prediction
        gather_predictions(data_for_predictions,

                           # Return predictions of:
                            models_q1$rgr,
                            models_q1$total_biomass,
                            models_q1$root_shoot_ratio_log)   %>%

            # Get fitted values
            pivot_wider(names_from = model, values_from = pred)  %>%
             rename_all(
              funs(

                # rename columns
                stringr::str_to_lower(.) %>%

                # Remove string from name and replace it with pred_
                stringr::str_replace(., c(string), "pred_") %>%

                # Remove dollar sing
                gsub("\\$", "", .))) %>%


        # Back transform log variables
        mutate(pred_root_shoot_ratio = exp(pred_root_shoot_ratio_log)) %>%

        # Select only pred variables
        dplyr::select(-c(init_height, pred_root_shoot_ratio_log))

In [ ]:
# Generate plot combinations
vars_q1 <-
  crossing(

    # Get all numeric variables to plot (all y)
    as_tibble(t(combn(dplyr::select(data_pred_biomass_growth, where(is.numeric)) %>% names, 1))),

    # Select factor variables to plot
    x_axis_var = dplyr::select(data_pred_biomass_growth, nfixer) %>%  names,
    group_var = dplyr::select(data_pred_biomass_growth, treatment) %>%  names)

In [ ]:
par(mfrow = c(2,2))
vars_q1 %>%
      # Gererate plots
      pmap(~boxplot_plot_pmap(data = data_pred_biomass_growth,
                        y = !!sym(..1), x = !!sym(..2),
                                fill = !!sym(..3)))